In [1]:
from pathlib import Path
import json
import requests

In [2]:
logical_models_folder = Path('../OMOPLogicalModels-manual')
mapping_files_folder = Path('omop_to_fhir_base')
serverBaseUrl = 'https://server.fire.ly'

# Load Logical Model, transform to Custom Resource and post to the server

This is the input the FHIR Mapping Engine needs, the process is documented here: https://docs.fire.ly/projects/FHIR-Mapper/transformsetup/logicalmodel.html

In [3]:
logical_model_patient = json.loads((logical_models_folder / 'OMOPPerson.LogicalModel.json').open().read())
custom_resource_patient = logical_model_patient.copy()

identifier = custom_resource_patient['url'].split('/')[-1]

# custom_resource_patient['url'] = 'http://hl7.org/fhir/StructureDefinition/'+identifier # Appears not to be necessary
custom_resource_patient['kind'] = 'resource'
custom_resource_patient['type'] = identifier
custom_resource_patient['id'] = identifier
custom_resource_patient['baseDefinition'] = 'http://hl7.org/fhir/StructureDefinition/DomainResource'
custom_resource_patient['differential']['element'][0]['type'].pop()

res = requests.put(f'{serverBaseUrl}/administration/StructureDefinition/{identifier}', json=custom_resource_patient)
print(res.status_code)
print(res.text)

200
{
  "resourceType": "StructureDefinition",
  "id": "OMOPPerson",
  "meta": {
    "versionId": "e0e8e5b1-931c-452b-8f2e-d08fbbcc157f",
    "lastUpdated": "2021-08-05T16:16:02.888+00:00"
  },
  "url": "https://fhir.ohdsi.org/LogicalModel/OMOPPerson",
  "name": "LogicalModel_OMOP_Person",
  "title": "LogicalModel OMOP Person",
  "status": "draft",
  "date": "2021-06-14T14:52:24.6891825Z",
  "description": "This table serves as the central identity management for all Persons in the database. It contains records that uniquely identify each person or patient, and some demographic information.",
  "fhirVersion": "4.0.1",
  "mapping": [
    {
      "identity": "OMOP-CDM-v5.3.1",
      "uri": "https://ohdsi.github.io/CommonDataModel/cdm531.html",
      "name": "OMOP Common Data Model v5.3.1"
    },
    {
      "identity": "OMOP-CDM-v6.0",
      "uri": "https://ohdsi.github.io/CommonDataModel/cdm60.html"
    }
  ],
  "kind": "resource",
  "abstract": false,
  "type": "OMOPPerson",
  "baseDef

# Post Concept Map to the server

This will be used to translate between Code Systems

In [4]:
concept_map_gender = json.loads((mapping_files_folder / 'omop_gender_to_administrative_gender.ConceptMap.json').open().read())
concept_map_id = concept_map_gender['id']
res = requests.put(f'{serverBaseUrl}/administration/ConceptMap/{concept_map_id}', json=concept_map_gender)
print(res.status_code)
print(res.text)

200
{
  "resourceType": "ConceptMap",
  "id": "OmopTest",
  "name": "OmopTest",
  "status": "draft",
  "group": [
    {
      "element": [
        {
          "target": [
            {
              "code": "female",
              "display": "Female",
              "equivalence": "equal"
            }
          ],
          "code": "F",
          "display": "FEMALE"
        },
        {
          "target": [
            {
              "code": "male",
              "display": "Male",
              "equivalence": "equal"
            }
          ],
          "code": "M",
          "display": "MALE"
        }
      ],
      "source": "https://fhir.ohdsi.org/CodeSystem/Gender",
      "target": "http://hl7.org/fhir/administrative-gender"
    }
  ],
  "url": "https://fhir.ohdsi.org/ConceptMap/OMOPPerson",
  "meta": {
    "versionId": "92a7b3ae-05c7-4535-963d-b0985c7f8901",
    "lastUpdated": "2021-08-05T16:16:03.956+00:00"
  }
}


# Post Structure Map to the server, via Mapping Language

The Mapping language will be translated to a Structure Map by the server, which will store it to be used as transformation between resource instances.

In [5]:
structure_map_omopperson_to_patient = (mapping_files_folder / 'OMOPPerson_to_FHIRPatient.map').open().read()
headers = {'Content-Type': "text/fhir-mapping; charset=utf-8", 'Accept': "application/fhir+json; fhirVersion=4.0"}
res = requests.post(f'{serverBaseUrl}/administration/$convert?persist=true', data=structure_map_omopperson_to_patient, headers=headers)
print(res.status_code)
print(res.text)

200
{
  "resourceType": "StructureMap",
  "id": "OMOPPerson",
  "url": "https://fhir.ohdsi.org/StructureMap/OMOPPerson",
  "name": "OMOPPerson",
  "title": "OMOPPerson",
  "status": "unknown",
  "experimental": false,
  "structure": [
    {
      "url": "https://fhir.ohdsi.org/LogicalModel/OMOPPerson",
      "mode": "source"
    },
    {
      "url": "http://hl7.org/fhir/StructureDefinition/Patient",
      "mode": "target"
    }
  ],
  "group": [
    {
      "name": "test",
      "typeMode": "none",
      "input": [
        {
          "name": "src",
          "type": "OMOPPerson",
          "mode": "source"
        },
        {
          "name": "tgt",
          "type": "Patient",
          "mode": "target"
        }
      ],
      "rule": [
        {
          "name": "rule-1",
          "source": [
            {
              "context": "src",
              "element": "person_id",
              "variable": "personId"
            }
          ],
          "target": [
            {
   

# Transform OMOP Logical Model instance to FHIR Patient instance

In [6]:
custom_resource_patient_instance = json.loads((mapping_files_folder / 'Example.OMOPPerson.json').open().read())

res = requests.post(f'{serverBaseUrl}/administration/StructureMap/{identifier}/$transform', json=custom_resource_patient_instance)
print(res.status_code)
print(res.text)

200
{
  "resourceType": "Patient",
  "identifier": [
    {
      "value": "1"
    }
  ],
  "gender": "female"
}
